In [37]:
# Import necessary libraries
import sys
sys.path.append('../src')  # Add the src directory to the system path

from data_loader import DataLoader
from data_clean_processing import DataCleanProcessing
import pandas as pd
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import chi2_contingency, ttest_ind
import statsmodels.api as sm
from statsmodels.stats.power import TTestIndPower, GofChisquarePower
from statsmodels.stats.contingency_tables import cochrans_q


In [3]:
# Load the data
file_path = '../data/MachineLearningRating_v3.txt'
data_loader = DataLoader(file_path)
data = data_loader.load_data()

In [4]:
# Display basic info and head of the data
data_loader.basic_info()
data_loader.display_head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Data columns (total 52 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   UnderwrittenCoverID       1000098 non-null  int64  
 1   PolicyID                  1000098 non-null  int64  
 2   TransactionMonth          1000098 non-null  object 
 3   IsVATRegistered           1000098 non-null  bool   
 4   Citizenship               1000098 non-null  object 
 5   LegalType                 1000098 non-null  object 
 6   Title                     1000098 non-null  object 
 7   Language                  1000098 non-null  object 
 8   Bank                      854137 non-null   object 
 9   AccountType               959866 non-null   object 
 10  MaritalStatus             991839 non-null   object 
 11  Gender                    990562 non-null   object 
 12  Country                   1000098 non-null  object 
 13  Province                  1

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [5]:
# Clean the data
data_cleaner = DataCleanProcessing(data)
cleaned_data = data_cleaner.clean_missing_values()
print("Verification of no missing values:", data_cleaner.verify_no_missing_values())


d:\Insurance_Claims_Analysis\notebooks\../src\data_clean_processing.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned[column] = data_cleaned[column].fillna(median)
d:\Insurance_Claims_Analysis\notebooks\../src\data_clean_processing.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned[column] = data_cleaned[column].fillna(median)
d:\Insurance_Claims_Analysis\notebooks\../src\data_clean_processing.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

Verification of no missing values: True


In [48]:
# Display the first few rows of the cleaned data
cleaned_data.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,Margin,RiskLevel,ProfitMargin
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,21.929825,0.0,1.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,21.929825,0.0,1.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,0.000000,0.0,NaN
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,512.848070,0.0,1.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,0.000000,0.0,NaN


In [22]:
print(cleaned_data.columns)

Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'AlarmImmobiliser', 'TrackingDevice',
       'CapitalOutstanding', 'NewVehicle', 'SumInsured', 'TermFrequency',
       'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory',
       'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass',
       'StatutoryRiskType', 'TotalPremium', 'TotalClaims', 'Margin'],
      dtype='object')


In [30]:
# Calculate the Margin
cleaned_data['Margin'] = cleaned_data['TotalPremium'] - cleaned_data['TotalClaims']

C:\Users\matiwos.desalegn\AppData\Local\Temp\ipykernel_27580\4030726266.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['Margin'] = cleaned_data['TotalPremium'] - cleaned_data['TotalClaims']


In [38]:
# Function to calculate Cramér's V
def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    return (chi2 / (n * (min(confusion_matrix.shape) - 1))) ** 0.5

In [39]:
# Function to perform power analysis for chi-squared test
def chi2_power_analysis(effect_size, alpha, power):
    analysis = GofChisquarePower()
    sample_size = analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power)
    return sample_size

In [40]:
# Function to perform power analysis for t-test
def ttest_power_analysis(effect_size, alpha, power):
    analysis = TTestIndPower()
    sample_size = analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power)
    return sample_size

In [41]:
# Hypothesis 1: Risk Differences Across Provinces
contingency_table_provinces = pd.crosstab(cleaned_data['Province'], cleaned_data['TotalClaims'])
chi2, p_provinces, dof, ex = chi2_contingency(contingency_table_provinces)
effect_size_provinces = cramers_v(contingency_table_provinces)
required_sample_provinces = chi2_power_analysis(effect_size_provinces, alpha=0.05, power=0.8)

In [42]:
# Hypothesis 2: Risk Differences Between Zipcodes
contingency_table_zipcodes = pd.crosstab(cleaned_data['PostalCode'], cleaned_data['TotalClaims'])
chi2, p_zipcodes, dof, ex = chi2_contingency(contingency_table_zipcodes)
effect_size_zipcodes = cramers_v(contingency_table_zipcodes)
required_sample_zipcodes = chi2_power_analysis(effect_size_zipcodes, alpha=0.05, power=0.8)

In [43]:
# Hypothesis 3: Margin Differences Between Zip Codes
group_a = cleaned_data[cleaned_data['CoverType'] == 'TypeA']
group_b = cleaned_data[cleaned_data['CoverType'] == 'TypeB']
group_a_margin = group_a['Margin']
group_b_margin = group_b['Margin']
t_stat, p_margin = ttest_ind(group_a_margin, group_b_margin, nan_policy='omit')
# Calculate effect size for t-test (Cohen's d)
mean_diff = group_a_margin.mean() - group_b_margin.mean()
pooled_sd = np.sqrt(((len(group_a_margin) - 1) * group_a_margin.std() ** 2 + (len(group_b_margin) - 1) * group_b_margin.std() ** 2) / (len(group_a_margin) + len(group_b_margin) - 2))
effect_size_margin = mean_diff / pooled_sd
required_sample_margin = ttest_power_analysis(effect_size_margin, alpha=0.05, power=0.8)

d:\Insurance_Claims_Analysis\.venv\Lib\site-packages\statsmodels\stats\power.py:524: ConvergenceWarning: 
Failed to converge on a solution.

  warnings.warn(convergence_doc, ConvergenceWarning)


In [44]:
# Hypothesis 4: Risk Differences Between Women and Men
contingency_table_gender = pd.crosstab(cleaned_data['Gender'], cleaned_data['TotalClaims'])
chi2, p_gender, dof, ex = chi2_contingency(contingency_table_gender)
effect_size_gender = cramers_v(contingency_table_gender)
required_sample_gender = chi2_power_analysis(effect_size_gender, alpha=0.05, power=0.8)

In [45]:
# Function to interpret p-values
def interpret_p_value(p, alpha=0.05):
    return "Reject Null Hypothesis" if p < alpha else "Fail to Reject Null Hypothesis"

# Results
results = {
    'Risk Differences Across Provinces': interpret_p_value(p_provinces),
    'Risk Differences Between Zipcodes': interpret_p_value(p_zipcodes),
    'Margin Differences Between Zip Codes': interpret_p_value(p_margin),
    'Risk Differences Between Women and Men': interpret_p_value(p_gender)
}

In [46]:
# Print the results
for hypothesis, result in results.items():
    print(f'{hypothesis}: {result}')

print(f'Hypothesis 1: Chi-squared test statistic: {chi2}, p-value: {p_provinces}, required sample size: {required_sample_provinces}')
print(f'Hypothesis 2: Chi-squared test statistic: {chi2}, p-value: {p_zipcodes}, required sample size: {required_sample_zipcodes}')
print(f'Hypothesis 3: t-test statistic: {t_stat}, p-value: {p_margin}, required sample size: {required_sample_margin}')
print(f'Hypothesis 4: Chi-squared test statistic: {chi2}, p-value: {p_gender}, required sample size: {required_sample_gender}')

Risk Differences Across Provinces: Fail to Reject Null Hypothesis
Risk Differences Between Zipcodes: Fail to Reject Null Hypothesis
Margin Differences Between Zip Codes: Fail to Reject Null Hypothesis
Risk Differences Between Women and Men: Fail to Reject Null Hypothesis
Hypothesis 1: Chi-squared test statistic: 2534.8814995125476, p-value: 1.0, required sample size: 5856.465923239249
Hypothesis 2: Chi-squared test statistic: 2534.8814995125476, p-value: 1.0, required sample size: 5734.44927291309
Hypothesis 3: t-test statistic: nan, p-value: nan, required sample size: [10.]
Hypothesis 4: Chi-squared test statistic: 2534.8814995125476, p-value: 1.0, required sample size: 6193.291242217724
